In [1]:
import os
import sys
os.chdir('../')
new_current_dir = os.getcwd()  # get new current working directory
print("New current directory:", new_current_dir)
sys.path.append('./src/python/')

# import
from preamble.preamble import *
from gamma.gamma import Gamma, GammaNPP
from gamma.common import npp_from_dict, set_of_frozenset, to_json, npp_from_json
from testing.result_processing import post_process_result
from prebuilt.a05_post_processing_result import main as post_processing_result

New current directory: /home/fiftyfour/Documents/NetworkPricingGraphContraction


In [4]:
# Sh prepare for original problem batch of 25
#############################################
# ./src/python/prebuilt/prebuilt/a01_prepare_sh_original.sh

In [2]:
# Time Config for original problems
directory_npp = './data/generated/problems/paper/original'
config = {}
result_dict  = {}
for root, dirs, files in os.walk(directory_npp):
    
    for filename in files:
        if filename.endswith("R.json"):
            
            base_name, ext = os.path.splitext(filename)
            _, _, *problem_name, _ = base_name.split('-')
            problem_name = '-'.join(problem_name)
            
            
            with open(os.path.join(root, filename), 'r') as f:
                result = json.load(f)

            config[problem_name] = (result['solve_time'], result['finish'])
            result_dict[problem_name] = result
with open('original_1000.pkl', 'wb') as f:
    pickle.dump(result_dict, f)

# Original result compile in one pkl
post_processing_result()
    

Processing original: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 199/199 [00:33<00:00,  5.88it/s]


In [14]:
# singular problem for testing from a pkl batch
# file_original = './data/generated/problems/paper/original/000000-000000-d40-08-P.json'
# file_problems = './data/generated/problems/paper/d40-08/000001-000000-000017-d40-08-P.pkl'
# file_transformations = './data/generated/problems/paper/d40-08/000001-000000-000017-d40-08-T.pkl'
# directory = './tmp/data/d40-08/'

# file_original = './data/generated/problems/paper/original/000000-000000-g30-10-P.json'
# file_problems = './tmp/data/g30-10/000001-000000-000096-g30-10-P.pkl'
# file_transformations = './tmp/data/g30-10/000001-000000-000096-g30-10-T.pkl'
# directory = './tmp/data/g30-10/'


file_original = './data/generated/problems/paper/original/000000-000000-d35-02-P.json'
file_problems = './data/generated/problems/paper/d35-02/000001-000000-000099-d35-02-P.pkl'
file_transformations = './data/generated/problems/paper/d35-02/000001-000000-000099-d35-02-T.pkl'



directory = './tmp/data/d35-02/'

with open(file_original, "r") as f:
    original = json.load(f)
    
with open(file_problems, "rb") as f:
    problems_ = pickle.load(f)
    
with open(file_transformations, "rb") as f:
    transformations_ = pickle.load(f)

k = list(transformations_.keys())[0]
print(k)

nodes, edges, problems = npp_from_dict(original)
transformations = transformations_[k]
gamma_ = GammaNPP.from_transformation(nodes, edges, transformations, problems=problems)

# extract one problem and transformation

file_p = k + '-P'
to_json(gamma_.image_problem_to_dict(), directory, file_p)
file_t = k + '-T'
to_json(gamma_.transformation_to_dict(), directory, file_t)
#

000069-100-2-0-5-1-1-1-1-1500-d35-02


'./tmp/data/d35-02/000069-100-2-0-5-1-1-1-1-1500-d35-02-T.json'

In [3]:
test_dict = gamma_.transformation_to_dict()

# print(test_dict['A'] == transformations['A'])
# print(test_dict['V'] == transformations['V'])
# print(set_of_frozenset(transformations['RV']) == set_of_frozenset(test_dict['RV']))
# print(test_dict['RA'] == transformations['RA'])
print(*list(filter(lambda x: len(x)>1, test_dict['RA'])), sep='\n')


(505, 292)
(119, 346, 324, 69, 335, 251)
(509, 100, 151, 161, 342, 487, 65)
(383, 198)
(514, 300)


In [13]:
def parameter_kwargs(n, min_sl, max_sl, m, H1, H2, H3, H4, max_attemp):
    return {
    'n':n, 'min_sl':min_sl, 'max_sl':max_sl, 'm':m,
    'H1':H1, 'H2':H2, 'H3':H3, 'H4':H4,
    'max_attemp':max_attemp
    }
    
# process result
result_dict = {}

problem_name = 'd40-08'
directory_original = './tmp/data/'
p = os.path.join(directory_original, f'000000-000000-{problem_name}-P.json')
t = os.path.join(directory_original, f'000000-000000-{problem_name}-T.json')
r = os.path.join(directory_original, f'000000-000000-{problem_name}-R.json')

o_nodes, o_edges, o_problems = npp_from_json(p)

ptr = {'000011-100-2-2-1-1-1-1-1-1500-d40-08':('./tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-P.json', './tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-T.json', './tmp/data/d40-08/000011-100-2-2-1-1-1-1-1-1500-d40-08-R.json')}

for batch_id, (p, t, r) in ptr.items():
    
    with open(t, 'rb') as f:
        transformations = json.load(f)
    
    with open(r, 'r') as f:
        results = json.load(f)
        results = {r.pop('id'): r for r in results}

    
    for id_ in tqdm(results.keys(), desc=f'Processing batch : {batch_id}'):
        # id_ ex. 000840-50-2-5-0-1-1-1-0-1500-d30-07
        
        _, *params, _ = id_.replace(problem_name, '').split('-')
        
        result_dict[id_] = post_process_result(
                    o_nodes, o_edges, o_problems,
                    transformations, results[id_],
                    **parameter_kwargs(100,2,2,1,1,1,1,1,1500)
                    )

Processing batch : 000011-100-2-2-1-1-1-1-1-1500-d40-08:   0%|                                                                         | 0/24 [00:00<?, ?it/s]

9


Processing batch : 000011-100-2-2-1-1-1-1-1-1500-d40-08:   4%|██▋                                                              | 1/24 [00:00<00:07,  2.92it/s]

11


Processing batch : 000011-100-2-2-1-1-1-1-1-1500-d40-08:   8%|█████▍                                                           | 2/24 [00:00<00:06,  3.41it/s]

...

12


Processing batch : 000011-100-2-2-1-1-1-1-1-1500-d40-08:  92%|██████████████████████████████████████████████████████████▋     | 22/24 [00:06<00:00,  3.62it/s]

12


Processing batch : 000011-100-2-2-1-1-1-1-1-1500-d40-08: 100%|████████████████████████████████████████████████████████████████| 24/24 [00:06<00:00,  3.60it/s]

12


In [14]:
df = pd.DataFrame(result_dict)

In [15]:
df

,000011-100-2-2-1-1-1-1-1-1500-d40-08-P,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-gamma-min,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-gamma-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-gamma-max,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-1,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-2,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-3,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-4,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-5,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-1-min,...,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-1-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-2-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-3-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-4-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-5-avg,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-1-max,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-2-max,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-3-max,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-4-max,000011-100-2-2-1-1-1-1-1-1500-d40-08-P-retro-5-max
edge,"[[5, 5, 23.0, 16], [7, 7, 0.0, 7], [10, 10, 3....","[[5, 5, 24.0, 16], [7, 7, 0.0, 7], [10, 10, 19...","[[5, 5, 8.0, 16], [7, 7, 0.0, 7], [10, 10, 0.0...","[[5, 5, 23.0, 16], [7, 7, 6.0, 7], [10, 10, 3....","[[5, 5, 23.0, 15], [7, 7, 0.0, 10], [10, 10, 3...","[[5, 5, 24.0, 15], [7, 7, 2.0, 10], [10, 10, 3...","[[5, 5, 23.0, 15], [7, 7, 0.0, 10], [10, 10, 3...","[[5, 5, 24.0, 15], [7, 7, 0.0, 10], [10, 10, 3...","[[5, 5, 23.0, 15], [7, 7, 0.0, 10], [10, 10, 3...","[[5, 5, 24.0, 15], [7, 7, 0.0, 10], [10, 10, 1...",...,"[[5, 5, 8.0, 15], [7, 7, 0.0, 10], [10, 10, 0....","[[5, 5, 33.0, 15], [7, 7, 0.0, 10], [10, 10, 0...","[[5, 5, 8.0, 15], [7, 7, 0.0, 10], [10, 10, 0....","[[5, 5, 24.0, 15], [7, 7, 6.0, 10], [10, 10, 3...","[[5, 5, 8.0, 15], [7, 7, 0.0, 10], [10, 10, 0....","[[5, 5, 23.0, 15], [7, 7, 6.0, 10], [10, 10, 3...","[[5, 5, 24.0, 15], [7, 7, 6.0, 10], [10, 10, 3...","[[5, 5, 23.0, 15], [7, 7, 6.0, 10], [10, 10, 3...","[[5, 5, 33.0, 15], [7, 7, 20.0, 10], [10, 10, ...","[[5, 5, 23.0, 15], [7, 7, 6.0, 10], [10, 10, 3..."
obj_value,95644.0,96838.0,96565.0,95553.0,91403.0,99667.0,94680.0,101189.000001,94680.0,90904.0,...,91565.0,99662.0,95164.0,103348.0,95164.0,85326.0,98303.0,94336.0,103017.0,94336.0
preprocess_time,2.94954,3.373251,3.381077,2.616497,2.989236,2.901777,2.319295,2.905027,2.96356,2.300401,...,3.024089,2.296137,2.990618,2.685592,3.100967,2.998107,2.260978,3.599605,2.821065,2.867581
solve_time,26.622199,25.102433,25.083253,25.123624,0.396057,11.325085,4.814395,20.019005,4.746263,0.373854,...,0.357698,7.072215,15.339115,12.244474,15.372856,0.357963,6.180157,15.740953,14.481172,16.133978
n_vertex,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143
n_edge,831,831,831,831,831,831,831,831,831,831,...,831,831,831,831,831,831,831,831,831,831
n_tolled,165,165,165,165,165,165,165,165,165,165,...,165,165,165,165,165,165,165,165,165,165
compression_factors,"{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...",...,"{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ...","{1: 1.0012033694344165, 2: 1.006993006993007, ..."
rewind_optimal_1,91403,90904,91565,8

In [16]:
with open(os.path.join('./tmp/', f'result_{problem_name}.pkl'), 'wb') as f:
    pickle.dump(result_dict, f)